In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [19]:
from sklearn.model_selection import train_test_split

In [7]:
import matplotlib.pylab as plt
%matplotlib inline

In [13]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


In [15]:
from keras.layers import BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.layers import Activation, Dense, Dropout, LSTM
from keras.optimizers import adam
from tensorflow.keras.optimizers import SGD

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


import datetime

## train data, label data 생성
라벨 데이터 생성 함수 실행 전에 label_512.csv 삭제 후 진행할 것. 조건문 활용 코드 수정 요망

In [8]:
#train data 생성
def load_train_data(train_data_num, train_data_path):
    
    #train_data_num=input('csv file 몇개?')
    #train_data_path=input('path?')
    filename = train_data_path+'/dataset_512_csv1.csv'
    train_data=np.loadtxt(filename,delimiter=',')
    
    for i in range(2,int(train_data_num)+1):
        filename = train_data_path + '/dataset_512_csv'+str(i)+'.csv'
        train_data=np.append(train_data,np.loadtxt(filename,delimiter=','),axis=0)
    
    frame = list(range(0,300))
    train_data_df = pd.DataFrame(train_data, columns=frame)

    return train_data_df

In [ ]:
#label 생성
def load_label_data():
    train_data_num = input('csv file 몇개?')
    train_data_path = input('path?')
    
    label_512=open(train_data_path + "/label_512.csv","a")
    csv_label = pd.read_csv(train_data_path + '/label.csv')
    label_512.write('label\n')

    for a in range(0, int(train_data_num)): #데이터 파일 갯수
        each_label = csv_label.iloc[a,0]
        for b in range(0, 512): #각 파일당 행 갯수(고정)
                 label_512.write(str(each_label))
                 label_512.write('\n')
    label_data_df = pd.read_csv(train_data_path + "/label_512.csv")
    
    return label_data_df

In [9]:
train_set1 = load_train_data(22, "C:/Users/DSPL/Desktop/딥러닝 스터디 발표/data/0804/dataset")

In [10]:
train_set2 = load_train_data(20, "C:/Users/DSPL/Desktop/딥러닝 스터디 발표/data/0731/dataset")

In [ ]:
label_1 = load_label_data()

In [ ]:
label_2 = load_label_data()

In [11]:
#생성 데이터들 합치기(train, test, validation 한번에 나누기 위함)
label_1 = pd.read_csv('C:/Users/DSPL/Desktop/딥러닝 스터디 발표/data/0804/dataset/label_512.csv') #자꾸 오류나서 그냥 다시 로드
label_2 = pd.read_csv('C:/Users/DSPL/Desktop/딥러닝 스터디 발표/data/0731/dataset/label_512.csv')

all_data = pd.concat([train_set1, train_set2], ignore_index=True)
all_label = pd.concat([label_1, label_2], ignore_index=True)

print(label_1.shape)
print(label_2.shape)
print(all_data.shape)
print(all_label.shape)

(11264, 1)
(10240, 1)
(21504, 300)
(21504, 1)


## train, validation, test set 분할

In [17]:
data = all_data
target = all_label

In [20]:
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.2, random_state=1)

data_train, data_val, target_train, target_val = train_test_split(data_train, target_train, test_size=0.2, random_state=1)


print(data_train.shape, data_test.shape, data_val.shape, target_train.shape, target_test.shape, target_val.shape)

(13762, 300) (4301, 300) (3441, 300) (13762, 1) (4301, 1) (3441, 1)


In [21]:
#(13762, 300) (4301, 300) (3441, 300) (13762, 1) (4301, 1) (3441, 1)

data_train = np.expand_dims(data_train,axis=-1)
data_test = np.expand_dims(data_test,axis=-1)
data_val = np.expand_dims(data_val, axis=-1)
print(data_train.shape, target_train.shape, data_val.shape)
n_timesteps, n_features, n_outputs =data_train.shape[1], data_train.shape[2],target_train.shape[1]


(13762, 300, 1) (13762, 1) (3441, 300, 1)


In [22]:
%load_ext tensorboard

## batch normalization 4th model

In [ ]:
#batch 네번째 
#padding 추가
model_batch_norm4 = tf.keras.Sequential()

#hidden1
model_batch_norm4.add(tf.keras.layers.Conv1D(filters=16, kernel_size=3, padding='valid', input_shape=(n_timesteps,n_features)))
model_batch_norm4.add(tf.keras.layers.BatchNormalization())
model_batch_norm4.add(tf.keras.layers.ReLU())    
model_batch_norm4.add(tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='valid'))# Add Batchnorm layer before Activation

   
model_batch_norm4.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='valid'))
model_batch_norm4.add(tf.keras.layers.BatchNormalization())                    # Add Batchnorm layer before Activation
model_batch_norm4.add(tf.keras.layers.ReLU())    
model_batch_norm4.add(tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    
    
model_batch_norm4.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='valid'))
model_batch_norm4.add(tf.keras.layers.BatchNormalization())                    # Add Batchnorm layer before Activation
model_batch_norm4.add(tf.keras.layers.ReLU())


model_batch_norm4.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='valid'))
model_batch_norm4.add(tf.keras.layers.BatchNormalization())                    # Add Batchnorm layer before Activation
model_batch_norm4.add(tf.keras.layers.ReLU())
    
    
model_batch_norm4.add(tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='valid'))
model_batch_norm4.add(tf.keras.layers.BatchNormalization())                    # Add Batchnorm layer before Activation
model_batch_norm4.add(tf.keras.layers.ReLU())
model_batch_norm4.add(tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='valid'))



model_batch_norm4.add(tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='valid'))
model_batch_norm4.add(tf.keras.layers.BatchNormalization())                    # Add Batchnorm layer before Activation
model_batch_norm4.add(tf.keras.layers.ReLU())



model_batch_norm4.add(tf.keras.layers.Flatten())
model_batch_norm4.add(tf.keras.layers.Dense(64, activation='relu'))
model_batch_norm4.add(tf.keras.layers.Dense(10, activation='relu'))
model_batch_norm4.add(tf.keras.layers.Dense(1, activation='relu'))

    
# Compile model
model_batch_norm4.compile(optimizer=keras.optimizers.Adam(lr=0.0001,epsilon=None, decay=0.0, amsgrad=False),
                  loss='mse', metrics=['mean_absolute_error'])

log_dir = "logs\\fit_batch\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model_batch_norm4.summary()

In [ ]:
import keras
import os
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils.vis_utils import plot_model
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot


keras.utils.vis_utils.pydot = pydot
tf.keras.utils.plot_model(model_batch_norm4, to_file='model_combined.png')

In [ ]:
history = model_batch_norm4.fit(x = data_train, y = target_train, batch_size=64, epochs=200, shuffle=True,
                    validation_data=(data_val, target_val), 
                    callbacks=[tensorboard_callback])

In [ ]:
#epoch 200에서 300으로 증가함.

history4_1 = model_batch_norm4.fit(x = data_train, y = target_train, batch_size=64, epochs=300, shuffle=True,
                    validation_data=(data_val, target_val), 
                    callbacks=[tensorboard_callback])

In [ ]:
score = model_batch_norm4.evaluate(data_test, target_test, batch_size=64)
print("test loss, mean_absolute_error:", score)

In [ ]:
score_2 = model_batch_norm4.evaluate(data_test, target_test, batch_size=64)
print("test loss, mean_absolute_error:", score_2)

In [ ]:
from keras.models import load_model
model_batch_norm4.save('model_batch_norm4_epoch300.h5')

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('model mean_absolute_error')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()